In [2]:
import pandas as pd
import seaborn as sns
import os

# Time and Pivot Tables: Additional Project  

## Files contain product sales through telemarketing. Each file includes at least 4 columns: FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

### The task is to verify product activations for specific users by merging sales files with system connection logs.  

### Data specifics:  

1. **Telemarketing employees do not always provide a full ID; if 'id' is missing at the beginning of SUBS_ID, it must be added.**  
2. **The fields in files may be arranged randomly, but field names are static.**  
3. **A sale is not counted if the disconnection (END_DTTM) occurs less than 5 minutes after activation (START_DTTM).**  
4. **If a row in the sales file has no SUBS_ID specified, it is skipped.**  
### Save the result to a CSV file with a semicolon (`;`) delimiter, containing only valid activations.  

In [3]:
path_to_files = ('D:/курс_архив/my_files/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/')

In [4]:
path_to_files

'D:/курс_архив/my_files/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/'

In [5]:
# Reading the file with product activation logs  
prod_activations_logs = pd.read_csv('D:курс_архив/my_files/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/prod_activations_logs.csv', sep=';')

In [6]:
# Reset the index to make it continuous  
prod_activations_logs = prod_activations_logs.reset_index(drop=True)

In [7]:
prod_activations_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
0,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
1,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43
5,id8641743,2752,20-03-2020 15:44,21-04-2020 15:44
6,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00
7,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42
8,id8741631,5677,19-03-2020 12:28,01-12-2020 00:00


In [8]:
# Function to add 'id' to the beginning of SUBS_ID if it's missing  
def add_id(x):
    if x.startswith('id'):
        x = x
    else:
        x = 'id' + x
    return x 

In [9]:
# Create an empty DataFrame to combine all sales files  
# Loop through all files in the folder  
# Check if the file starts with "tm" (sales files)  
# Read each sales file and remove rows with empty SUBS_ID  
# Combine the current file with the overall DataFrame  
# Reset the indexes after each merge  
# Apply the add_id function to all values in the SUBS_ID column  
full_df = pd.DataFrame()
for i in os.listdir(path_to_files):
    if i.startswith("tm"):
        df_tm = pd.read_csv(path_to_files + i,sep=';').dropna(subset=["SUBS_ID"])
        full_df = pd.concat([full_df, df_tm])
        full_df = full_df.reset_index(drop=True)
        full_df['SUBS_ID'] = full_df.SUBS_ID.apply(add_id)    

In [9]:
# Verify the final DataFrame  
full_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,id8362218,7,9879,05-03-2020 11:42
5,id2185490,2,3210,16-03-2020 16:28
6,id5764122,3,1499,18-03-2020 15:44
7,id7642700,6,3020,15-03-2020 14:21
8,id1374509,2,5677,17-03-2020 11:48


In [10]:
# Check data types in the table  
full_df.dtypes

SUBS_ID      object
FILIAL_ID     int64
PROD_ID       int64
ACT_DTTM     object
dtype: object

In [11]:
# Merge the sales DataFrame and connection logs using SUBS_ID and PROD_ID as keys  
full_df = full_df.merge(prod_activations_logs, how='inner', on=['SUBS_ID', 'PROD_ID'])

In [12]:
# Check data types in the table again  
full_df.dtypes

SUBS_ID       object
FILIAL_ID      int64
PROD_ID        int64
ACT_DTTM      object
START_DTTM    object
END_DTTM      object
dtype: object

In [13]:
# Convert START_DTTM and END_DTTM columns to datetime format  
full_df['START_DTTM'] = pd.to_datetime(full_df.START_DTTM, format='%d-%m-%Y %H:%M')

In [14]:
# Convert START_DTTM and END_DTTM columns to datetime format  
full_df['END_DTTM'] = pd.to_datetime(full_df.END_DTTM, format='%d-%m-%Y %H:%M')

In [15]:
# Check data types after conversion  
full_df.dtypes

SUBS_ID               object
FILIAL_ID              int64
PROD_ID                int64
ACT_DTTM              object
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
dtype: object

In [16]:
# Display the DataFrame for review  
full_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00
1,id7646509,5,6431,19-03-2020 13:00,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00


In [17]:
# Create a new column `difference` containing the time difference between END_DTTM and START_DTTM  
full_df['difference'] = full_df.END_DTTM - full_df.START_DTTM

In [18]:
full_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id7646509,5,6431,19-03-2020 13:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [19]:
# Sort the DataFrame by the SUBS_ID column  
full_df.sort_values('SUBS_ID')

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
1,id7646509,5,6431,19-03-2020 13:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00


In [20]:
# Apply a filter: keep only rows where the difference is greater than 5 minutes  
filtered_df = full_df[full_df['difference'] > pd.Timedelta('5 minutes')]

In [21]:
filtered_df.sort_values('difference', ascending=False)

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
